In [4]:
from __future__ import print_function
from ROOT import TFile
import ROOT
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from math import sqrt,cos,acos,pi,exp,sin,atan2,log
from numpy import arange,mean,asarray,isnan,isinf
import pickle
from scipy import stats

from numpy import mean,std,asarray,vstack
from scipy.stats import gaussian_kde
import csv
import sys
import pandas as pd
import os
import sklearn

from SelectionHelper import BDTensemble,selection

from xgboost import XGBClassifier

from SelectionDefs import NewAng, VtxInSimpleFid, VtxInFid, GetPhiT, pTrans,pTransRat
from SelectionDefs import alphaT, ECCQE, ECal, Q2, OpenAngle, PhiDiff, edgeCut
from SelectionDefs import ECCQE_mom, Boost, BoostTracks, Getpz, Getq3q0

from BDTHelper import getNewShowerCalibTrainingVarbs,getNewShowerCalibDPFvarbs


fset = 12 # 0 = open data, 1-7 = fake datasets, 8 = High E far sideband, 9 = Low BDT far sideband, 10 = Final Sideband Process 1, 11 = Final Sideband Process 2
cutMode = 0 # 0 = Final Selection, 1 = Kin Cut Sample, 2 = High E, 3 = Low BDT
if fset==8: cutMode = 2
if fset==9: cutMode = 3
    
savedir = {0:'FinalSelection',1:'KinCut',2:'HighE',3:'LowBDT'}[cutMode]

# POT Breakdown for Each Run
C1_POT = 1.558e+20 + 1.129e+17 + 1.869e+19
D2_POT = 1.63e+20 + 2.964e+19 + 1.239e+19
E1_POT = 5.923e+19
F1_POT = 4.3e+19
G1_POT = 1.701e+20 + 2.97e+19 + 1.524e+17



inDat1 = None
inDat2 = None
inDat3 = None
# load in specialized data sets -I will need to update this for the open data eventually

nick_data_dir = "/home/kmason/fullosc/NicksCode/1e1pdata/"

if fset==0: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_v28_wctagger_5e19.root")
    mpid_rerun_dat_1 = np.empty(0)
    inDat3 = TFile.Open(nick_data_dir+"mcc9_v28_wctagger_run3_bnb1e19.root")
    mpid_rerun_dat_3 = np.empty(0)
    DAT_POT1 = 4.403e19
    DAT_POT2 = 0
    DAT_POT3 = 8.786e+18
elif fset==1: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set1_run1_dlana.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_fakedata_set1_run3b_dlana.root")
    DAT_POT1  = 1.935765e20
    DAT_POT2 = 0
    DAT_POT3  = 2.99974063e20
elif fset==2: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set2_run1_dlana.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_fakedata_set2_run3b_dlana.root")
    DAT_POT1 = 4.0129213616e20
    DAT_POT2 = 0
    DAT_POT3 = 3.85169302616e20
elif fset==3: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set3_run1_dlana.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_fakedata_set3_run3b_dlana.root")
    DAT_POT1 = 4.024614415411e20
    DAT_POT2 = 0
    DAT_POT3 = 3.9685728933181e20
elif fset==4: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set4_run1_dlana.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_fakedata_set4_run3b_dlana.root")
    DAT_POT1 = 3.907905653e20
    DAT_POT2 = 0
    DAT_POT3 = 3.92356759e20
elif fset==5: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set5_run1_dlana.root")
    inDat3 = None
    DAT_POT1 = 7.00587e20
    DAT_POT2 = 0
    DAT_POT3 = 0
elif fset==7: 
    inDat1 = TFile.Open(nick_data_dir+"mcc9_fakedata_set7_run1_dlana.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_fakedata_set7_run3_dlana.root")
    DAT_POT1 = 1.8382999360024655e+20
    DAT_POT2 = 0
    DAT_POT3 = 2.05E+20
elif(fset==8):
    inDat1 = TFile.Open(nick_data_dir+"highE_total_C1_D2_E1_F1_G1.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = C1_POT
    DAT_POT2 = D2_POT+E1_POT
    DAT_POT3 = F1_POT+G1_POT
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==9):
    inDat1 = TFile.Open(nick_data_dir+"lowBDT_total_C1_D2_E1_G1.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = C1_POT
    DAT_POT2 = D2_POT+E1_POT
    DAT_POT3 = G1_POT
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==10):
    inDat1 = TFile.Open(nick_data_dir+"nu1e1p_near_and_far_sideband_20210623.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.631e+20
    DAT_POT2 = 2.750e+20
    DAT_POT3 = 2.291e+20
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==11):
    inDat1 = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run1_nearbox_fvv_20210721.root")
    inDat2 = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run2_nearbox_fvv_20210721.root")
    inDat3 = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3_nu1e1p_near_sideband_dlana_v1_1_4_fvv_20210715.root")
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.69E+20
    DAT_POT2 = 2.73E+20
    DAT_POT3 = 2.44E+20
    
elif(fset==12):
    inDat1 = TFile.Open(nick_data_dir+"openbox/test_run123_1e1p_signal_fvv.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.631e+20
    DAT_POT2 = 2.750e+20
    DAT_POT3 = 2.291e+20
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3

    
DATA_POT_TOT = DAT_POT1 + DAT_POT2 + DAT_POT3

print(DAT_POT1,DAT_POT2,DAT_POT3)
print(DAT_POT1+DAT_POT2+DAT_POT3)
POTDICT = {1:DAT_POT1,2:DAT_POT2,3:DAT_POT3}


if inDat1 is not None: 
    if fset==11: inDatFF1 = inDat1.Get("FinalVertexVariables")
    else: inDatFF1 = inDat1.Get("dlana/FinalVertexVariables")
else: inDatFF1 = None
if inDat2 is not None:
    if fset==11 : inDatFF2 = inDat2.Get("FinalVertexVariables")
    else: inDatFF2 = inDat2.Get("dlana/FinalVertexVariables")
else: inDatFF2 = None
if inDat3 is not None: inDatFF3 = inDat3.Get("dlana/FinalVertexVariables")
else: inDatFF3 = None


6.672e+20 2.75e+20 2.291e+20
1.1713e+21


In [5]:
# load in MC files
# ---------------- Run 1 -------------- #

inExt1      = TFile.Open(nick_data_dir+"mcc9_v28_wctagger_extbnb_finalbdt.root")
inOvr1      = TFile.Open(nick_data_dir+"run1_overlay_stripmine.root")
inNue1      = TFile.Open(nick_data_dir+"run1_intrinsics_rerun_stripmine.root")
inNueLow1   = TFile.Open(nick_data_dir+"mcc9_v29e_run1_bnb_intrinsic_nue_overlay_LowE.root")
inNumuLow1  = TFile.Open(nick_data_dir+"mcc9_v29e_run1_bnb_nu_overlay_LowE.root")
inNCPi01    = TFile.Open(nick_data_dir+"mcc9_v40_NCPi0_run1_dlana_stripped_v2.root")
inCCPi01    = TFile.Open(nick_data_dir+"mcc9_v40_CCPi0_run1_dlana_stripped_v2.root") 
inFullOsc1    = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run1_fullosc_v1_1_3_fvv.root") 
    

inExtFF1    = inExt1.Get("dlana/FinalVertexVariables")
inOvrFF1    = inOvr1.Get("FinalVertexVariables")
inNueFF1    = inNue1.Get("FinalVertexVariables")
inNueLowFF1 = inNueLow1.Get("dlana/FinalVertexVariables")
inNumuLowFF1= inNumuLow1.Get("dlana/FinalVertexVariables")
inCCPi0FF1    = inCCPi01.Get("dlana/FinalVertexVariables")
inNCPi0FF1    = inNCPi01.Get("dlana/FinalVertexVariables")
inFullOscFF1    = inFullOsc1.Get("dlana/FinalVertexVariables")
#inDirtFF   = inDirt.Get("dlana/FinalVertexVariables") 



# ---------------- Run 2 -------------- #

inOvr2      = TFile.Open(nick_data_dir+"run2_overlay_stripmine.root")
inNue2      = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run2_bnb_intrinsics_nue_overlay_finalbdt.root")
inNueLow2   = TFile.Open(nick_data_dir+"mcc9_intrinsic_nue_lowe_run2_fvv_stripped.root")
inNumuLow2  = TFile.Open(nick_data_dir+"mcc9_bnb_overlay_lowe_run2_fvv_stripped.root")
inFullOsc2  = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3b_fullosc_v1_1_3_fvv.root")
    

inOvrFF2    = inOvr2.Get("FinalVertexVariables")
inNueFF2    = inNue2.Get("dlana/FinalVertexVariables")
inNueLowFF2 = inNueLow2.Get("dlana/FinalVertexVariables")
inNumuLowFF2= inNumuLow2.Get("dlana/FinalVertexVariables")
inFullOscFF2= inFullOsc2.Get("dlana/FinalVertexVariables")



# ---------------- Run 3 -------------- #

inExt3      = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3_G1_extbnb_finalbdt.root")
inOvr3      = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge_finalbdt.root")
inNue3      = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3b_bnb_intrinsic_nue_overlay_nocrtremerge_finalbdt.root")
# inNueLow3   = TFile.Open(nick_data_dir+"mcc9_v29e_dl_run3b_intrinsic_nue_LowE.root")
inNumuLow3  = TFile.Open(nick_data_dir+"mcc9_v29e_run3b_bnb_nu_overlay_LowE.root")
inNCPi03    = TFile.Open(nick_data_dir+"mcc9_v40_NCPi0_run3_dlana_stripped_v2.root")
inCCPi03    = TFile.Open(nick_data_dir+"mcc9_v40_bnb_nu_overlay_run3_CCPi0_nu1e1p_signal_v1_1_3_fvv.root")
    
           
inExtFF3    = inExt3.Get("dlana/FinalVertexVariables")
inOvrFF3    = inOvr3.Get("dlana/FinalVertexVariables")
inNueFF3    = inNue3.Get("dlana/FinalVertexVariables")
# inNueLowFF3 = inNueLow3.Get("dlana/FinalVertexVariables")
inNumuLowFF3 = inNumuLow3.Get("dlana/FinalVertexVariables")
inCCPi0FF3    = inCCPi03.Get("dlana/FinalVertexVariables")
inNCPi0FF3    = inNCPi03.Get("dlana/FinalVertexVariables")
inFullOscFF3    = inFullOsc2.Get("dlana/FinalVertexVariables")

In [11]:
# first apply all genie and pi0 weights
GENIE_tune = 2
apply_pi0_weights = True

weightdir = nick_data_dir+'aux/'

pi0w_numu_1 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_numu_run1.txt","r")
pi0w_nue_1 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_nue_run1.txt","r")
pi0w_nue_low_1 = None
pi0w_numu_low_1 = None
pi0w_fullosc_1 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_fullosc_run1.txt","r")
pi0w_ncpi0_1 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_ncpi0_run1.txt","r")
pi0w_ccpi0_1 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_ccpi0_run1.txt","r")

pi0w_numu_2 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_numu_run2.txt","r")
pi0w_nue_2 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_nue_run2.txt","r")
pi0w_numu_low_2 = None
pi0w_nue_low_2 = None

pi0w_numu_3 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_numu_run3.txt","r")
pi0w_nue_3 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_nue_run3.txt","r")
pi0w_nue_low_3 = None
pi0w_numu_low_3 = None
pi0w_fullosc_3 = None
# pi0w_fullosc_3 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_fullosc_run3.txt","r")
pi0w_ncpi0_3 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_ncpi0_run3.txt","r")
pi0w_ccpi0_3 = open(weightdir+"1e1p_pi0weights/fullosc/pi0_weights_1e1p_avgscore_ccpi0_run3.txt","r")


    
if GENIE_tune==1:
    ovrCVFile1  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run1.root") 
    nueCVFile1  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run1.root") 
    nueLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_lowE_run1.root")
    numuLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_lowE_run1.root")
    ovrCVFile2  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run2.root") 
    nueCVFile2  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run2.root")
    ovrCVFile3  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run3.root") 
    nueCVFile3  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run3.root") 
    nueLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_lowE_run3.root")
    numuLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_lowE_run3.root")    
elif GENIE_tune==2:
    ovrCVFile1  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run1.root") 
    nueCVFile1  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run1.root") 
    nueLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run1.root")
    numuLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run1.root")
    ovrCVFile2  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run2.root") 
    nueCVFile2  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run2.root")
    ovrCVFile3  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run3.root") 
    nueCVFile3  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run3.root") 
    nueLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run3.root")
    numuLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run3.root")
    
nueLowCVFile2 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run2.root")
numuLowCVFile2 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run2.root")
NCPi0CVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_nc_pi0_run1.root")
NCPi0CVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_nc_pi0_run3.root")
CCPi0CVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_cc_pi0_run1.root")
CCPi0CVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_cc_pi0_run3.root")
FullOscFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_fullosc_nue_run1.root")
FullOscFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_fullosc_nue_run3.root")    
    
ovrCV1      = ovrCVFile1.Get("eventweight_tree")
nueCV1      = nueCVFile1.Get("eventweight_tree")
nueLowCV1   = nueLowCVFile1.Get("eventweight_tree")
numuLowCV1  = nueLowCVFile1.Get("eventweight_tree")
ovrCV2      = ovrCVFile2.Get("eventweight_tree")
nueCV2      = nueCVFile2.Get("eventweight_tree")
nueLowCV2   = nueLowCVFile2.Get("eventweight_tree")
numuLowCV2  = nueLowCVFile2.Get("eventweight_tree")
ovrCV3      = ovrCVFile3.Get("eventweight_tree")
nueCV3      = nueCVFile3.Get("eventweight_tree")
nueLowCV3   = nueLowCVFile3.Get("eventweight_tree")
numuLowCV3  = numuLowCVFile3.Get("eventweight_tree")

ccpi0_1     = CCPi0CVFile1.Get("eventweight_tree")
ccpi0_3     = CCPi0CVFile3.Get("eventweight_tree")
ncpi0_1     = NCPi0CVFile1.Get("eventweight_tree")
ncpi0_3     = NCPi0CVFile3.Get("eventweight_tree")

fullosc_1     = FullOscFile1.Get("eventweight_tree")
fullosc_3     = FullOscFile3.Get("eventweight_tree")

ovrCVDict = {}
nueCVDict = {}
nueLowCVDict = {}
numuLowCVDict = {}
leeCVDict = {}
leeLowCVDict = {}

ncpi0CVDict = {}
ccpi0CVDict = {}

fulloscCVDict = {}

def applyeventweights(eventdict, eventtree, pi0weights, LEEdict, useLEE = False, apply_pi0_weights = True):
    for x in eventtree:
        idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
        weight = x.xsec_corr_weight
        if isnan(weight) or isinf(weight):
            eventdict[idx] = 1.0
        else:
            eventdict[idx] = weight
            
        if useLEE:
            if isnan(leeweight) or isinf(leeweight):
                leeCVDict[idx] = 0.0
            else:
                leeCVDict[idx] = leeweight

    if apply_pi0_weights and pi0weights is not None:
        all_data = [(line.strip()).split(',') for line in pi0weights.readlines()]
        data = all_data[:]
        for line in data:
            print(line)
            if line[0]=='run': continue
            r = int(line[0])
            s = int(line[1])
            e = int(line[2])
            E = round(float(line[3]),0)
            w = float(line[-1])
            if w>0:
                for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
                    if idx in eventdict: 
                        eventdict[idx]*=w
                        continue
        pi0weights.close()
    return


print('run 1 ccpi0')
applyeventweights(ccpi0CVDict, ccpi0_1, pi0w_ccpi0_1,leeCVDict,False,True)
       
print('run 3 ccpi0')
applyeventweights(ccpi0CVDict, ccpi0_3, pi0w_ccpi0_3,leeCVDict,False,True)
print(ccpi0CVDict)
        
print('run 1 ncpi0')
applyeventweights(ncpi0CVDict, ncpi0_1, pi0w_ncpi0_1,leeCVDict,False,True)
        
print('run 3 ncpi0')
applyeventweights(ncpi0CVDict, ncpi0_3, pi0w_ncpi0_3,leeCVDict,False,True)

# ---------------- Run 1 -------------- #
print('run 1 bnb nu')
applyeventweights(ovrCVDict, ovrCV1, pi0w_numu_1,leeCVDict,False,True)
        
print('run 1 bnb nu lowE')
applyeventweights(numuLowCVDict, numuLowCV1, pi0w_numu_low_1,leeCVDict,False,True)

print('run 1 intrinsic nue')
applyeventweights(nueCVDict, nueCV1, pi0w_nue_1,leeCVDict,True,True)
      
print('run 1 intrinsic nue lowE')
applyeventweights(nueLowCVDict, nueLowCV1, pi0w_nue_low_1,leeCVDict,True,True)
    
print('run 1 fullosc')
applyeventweights(fulloscCVDict, fullosc_1, pi0w_fullosc_1,leeCVDict,False,True) 

# ---------------- Run 2 -------------- #
print('run 2 bnb nu')
applyeventweights(ovrCVDict, ovrCV2, pi0w_numu_2,leeCVDict,False,True)
    
print('run 2 bnb nu lowE')
applyeventweights(numuLowCVDict, numuLowCV2, pi0w_numu_low_2,leeCVDict,False,True)

print('run 2 intrinsic nue')
applyeventweights(nueCVDict, nueCV2, pi0w_nue_2,leeCVDict,True,True)
    
print('run 2 intrinsic nue lowE')
applyeventweights(nueLowCVDict, nueLowCV2, pi0w_nue_low_2,leeCVDict,True,True)

# ---------------- Run 3 -------------- #
print('run 3 bnb nu')
applyeventweights(ovrCVDict, ovrCV3, pi0w_numu_3,leeCVDict,False,True)
        
print('run 3 bnb nu lowE')
applyeventweights(numuLowCVDict, numuLowCV3, pi0w_numu_low_3,leeCVDict,False,True)

print('run 3 intrinsic nue')
applyeventweights(nueCVDict, nueCV3, pi0w_nue_3,leeCVDict,True,True)
      
print('run 3 intrinsic nue lowE')
applyeventweights(nueLowCVDict, nueLowCV3, pi0w_nue_low_3,leeCVDict,True,True)
    
print('run 3 fullosc')
applyeventweights(fulloscCVDict, fullosc_3, pi0w_fullosc_3,leeCVDict,False,True) 

run 1 ccpi0
['7645', '138', '6911', '0.0043644784', '799.5737275494923', '28.160860061645508']
['7628', '133', '6665', '0.00881647', '335.1417504285723', '90.80158996582031']
['7504', '16', '812', '0.012953634', '221.25712680062614', '10.255046844482422']
['7388', '17', '870', '0.49383458', '1389.0285545211998', '58.51188278198242']
['7145', '8', '439', '0.0054526376', '332.4193269289879', '29.72397232055664']
['7055', '690', '34520', '0.01404281', '412.19028058123394', '62.96121597290039']
['7055', '490', '24522', '0.007628596', '156.52185467665566', '233.580322265625']
['7055', '457', '22859', '0.06675812', '95.92177057490213', '9.922806739807129']
['7055', '174', '8738', '0.057771873', '331.5301998670442', '26.45591163635254']
['7055', '174', '8713', '0.010982679', '927.6614320318205', '66.61690521240234']
['7054', '1944', '97249', '0.0026121296', '199.64319678949502', '13.56799030303955']
['7054', '1944', '97205', '0.0023829762', '457.38963985568284', '150.2992401123047']
['7054', 

run 3 ncpi0
run 1 bnb nu
run 1 bnb nu lowE
run 1 intrinsic nue
run 1 intrinsic nue lowE
run 1 fullosc


NameError: name 'pi0w_fullosc1' is not defined

In [4]:
from sklearn import preprocessing
#tag='7July2021_vA_fullLowE_withPi0Sample_newShowerCalib_noEnu'
# tag='3March2021_vA_fullLowE_withPi0Sample_newShowerCalib'
tag='23Aug2021_vA_fullLowE_withPi0Sample_newShowerCalib'
BDTnumlist=[i for i in range(20)]
oldBDT = False

# bdtsavedir = '/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/'
# bdt = {1:{},2:{},3:{}}
# for r in [1,2,3]:
#     for b in BDTnumlist:
#         print(b,r)
#         bdt[r][b] = pd.read_pickle(bdtsavedir+'BDTweights_R%i_%i.pickle'%(r,b))
#     print(bdt[r])

ensemble = BDTensemble(tag,BDTnumlist,oldBDT=oldBDT)


/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R1_0.pickle rb
3452 3452


/home/kmason/.local/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.4 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


13052 13051
628 628
0 2190
0 557
1657 1657
2394 2393
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R1_1.pickle rb
3452 3452


/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R1_0_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R1_0_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)


13052 13051
628 628
0 2190
0 557
1657 1657
2394 2393
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R1_2.pickle rb
3452 3452


...

/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_15_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_15_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_16_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/Ni

1991 1990
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R3_16.pickle rb
1340 1340
8772 8772
1150 1149
0 2159
0 488
1384 1383
1991 1990
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R3_17.pickle rb
1340 1340
8772 8772
1150 1149
0 2159
0 488
1384 1383
1991 1990
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R3_18.pickle rb
1340 1340


/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_17_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_17_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_18_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/Ni

8772 8772
1150 1149
0 2159
0 488
1384 1383
1991 1990
Exception
Exception
/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/BDTweights_R3_19.pickle rb
1340 1340
8772 8772
1150 1149
0 2159
0 488
1384 1383
1991 1990
Exception
Exception


/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_19_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
/home/kmason/fullosc/1e1pBDTCode/SelectionHelper.py:323: UserWarning: loadtxt: Empty input file: "/home/kmason/fullosc/NicksCode/1e1pdata/aux/BDTWeights/ValSample_R3_19_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)


In [5]:
### All possible cuts on data/mc ###
lowEpatch      = False#True if fset in [0,5,8,9] else False
    

In [6]:
# add full osc ones here

filetaglist = {1:['data','ext','overlay','overlay_lowE','intrinsics','intrinsics_lowE','ncpi0','ccpi0','fullosc'],
               2:['data','overlay','overlay_lowE','intrinsics','intrinsics_lowE','fullosc'],
               3:['data','ext','overlay','overlay_lowE','intrinsics','ncpi0','ccpi0','fullosc']}

tlist = {1:[inDatFF1,inExtFF1,inOvrFF1,inNumuLowFF1,inNueFF1,inNueLowFF1,inNCPi0FF1,inCCPi0FF1,inFullOscFF1],
         2:[inDatFF2,inOvrFF2,inNumuLowFF2,inNueFF2,inNueLowFF2,inFullOscFF2],
         3:[inDatFF3,inExtFF3,inOvrFF3,inNumuLowFF3,inNueFF3,inNCPi0FF3,inCCPi0FF3,inFullOscFF3]}

GENIElist = {1:[None,None,ovrCVDict,numuLowCVDict,nueCVDict,nueLowCVDict,ncpi0CVDict,ccpi0CVDict,fulloscCVDict],
             2:[None,ovrCVDict,numuLowCVDict,nueCVDict,nueLowCVDict,fulloscCVDict],
             3:[None,None,ovrCVDict,numuLowCVDict,nueCVDict,ncpi0CVDict,ccpi0CVDict,fulloscCVDict]}

LEElist = {1:[None,None,None,None,leeCVDict,leeLowCVDict,None,None,None],
           2:[None,None,None,leeCVDict,leeLowCVDict,None],
           3:[None,None,None,None,leeCVDict,None,None,None]}

psavedir = 'Parquets/'+tag+'/'
try:
    os.mkdir(psavedir)
except OSError as error:
    print(error)

for r in [1,2,3]:
    print('\n\n','#'*20,'RUN',r,'#'*20,'\n')

    for t,filetag,genieDict,leeDict in zip(tlist[r],
                                           filetaglist[r],
                                           GENIElist[r],
                                           LEElist[r]):
        if not lowEpatch and 'lowE' in filetag: continue
        if t is None: continue
        if 'data' in filetag: continue
        print('Sample:',filetag)
        if 'data' in filetag:
            df = selection(t,cutMode,filetag,r,lowEpatch,ensemble,POT=POTDICT[r])
            pstring = 'SelectionMode%i_%s_fset%i_run%i.parquet.gzip'%(cutMode,filetag,fset,r)
            print(pstring)
        else:
            df = selection(t,cutMode,filetag,r,lowEpatch,ensemble,genieDict=genieDict,leeDict=leeDict)
            pstring = 'SelectionMode%i_%s_fset%i_run%i.parquet.gzip'%(cutMode,filetag,fset,r)
        df.to_parquet(psavedir+pstring,compression='gzip')

[Errno 17] File exists: 'Parquets/23Aug2021_vA_fullLowE_withPi0Sample_newShowerCalib/'


 #################### RUN 1 #################### 

Sample: ext
length check 263
length check 263
length test 263
Sample: overlay
length check 5602
length check 5602
length test 5602
Sample: intrinsics
length check 14733
length check 14733
length test 14733
Sample: ncpi0
length check 589
length check 589
length test 589
Sample: ccpi0
length check 285
length check 285
length test 285
Sample: fullosc
length check 7011
length check 7011
length test 7011


 #################### RUN 2 #################### 

Sample: overlay
length check 5784
length check 5784
length test 5784
Sample: intrinsics
length check 11241
length check 11241
length test 11241
Sample: fullosc
length check 7183
length check 7183
length test 7183


 #################### RUN 3 #################### 

Sample: ext
length check 537
length check 537
length test 537
Sample: overlay
length check 3943
length check 3943
length test 3943
Sample: